# Gardenpath Sentence on AllenNLP SRL BERT

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
import csv

In [4]:
from checklist.pred_wrapper import PredictorWrapper

In [5]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [6]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("John painted the wall with cracks.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'painted',
   'description': '[ARG0: John] [V: painted] [ARG1: the wall] [ARG2: with cracks] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'O']}],
 'words': ['John', 'painted', 'the', 'wall', 'with', 'cracks', '.']}

In [7]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [8]:
d = ["John painted the wall with cracks."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'painted',
     'description': '[ARG0: John] [V: painted] [ARG1: the wall] [ARG2: with cracks] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'B-ARG2', 'I-ARG2', 'O']}],
   'words': ['John', 'painted', 'the', 'wall', 'with', 'cracks', '.']}],
 array([1.]))

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [10]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [11]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [12]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [13]:
object = ['the wall with cracks', 'the plate with flowers', 'the table with cracks', 'the ceiling with cracks']

In [14]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    object = set([meta['object']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == object:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [15]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} painted {object}.", object = object, meta=True,nsamples=100, remove_duplicates=True)
# initialize a test object
with open('/datasets/gardenpath_bert.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/gardenpath_bert.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 100 examples
Test cases:      100
Fails (rate):    100 (100.0%)

Example fails:
[ARG0: Amy] [V: painted] [ARG1: the ceiling] [ARG2: with cracks] .
----
[ARG0: Bob] [V: painted] [ARG1: the wall] [ARG2: with cracks] .
----
[ARG0: Colin] [V: painted] [ARG1: the plate] [ARG2: with flowers] .
----
['Bobby', 'painted', 'the', 'ceiling', 'with', 'cracks', '.'] [False]
['Nick', 'painted', 'the', 'wall', 'with', 'cracks', '.'] [False]
['Suzanne', 'painted', 'the', 'plate', 'with', 'flowers', '.'] [False]
['Emily', 'painted', 'the', 'table', 'with', 'cracks', '.'] [False]
['Laura', 'painted', 'the', 'table', 'with', 'cracks', '.'] [False]
['Katherine', 'painted', 'the', 'ceiling', 'with', 'cracks', '.'] [False]
['Martin', 'painted', 'the', 'table', 'with', 'cracks', '.'] [False]
['Victoria', 'painted', 'the', 'table', 'with', 'cracks', '.'] [False]
['Bruce', 'painted', 'the', 'wall', 'with', 'cracks', '.'] [False]
['Susan', 'painted', 'the', 'plate', 'with', 'flowers', '.'] [False]
['

# End of Notebook